In [1]:
# 파이썬 2와 파이썬 3 지원
from __future__ import division, print_function, unicode_literals

# 공통
import numpy as np
import os

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# 맷플롯립 설정
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
plt.rcParams['font.family'] = 'NanumBarunGothic'
plt.rcParams['axes.unicode_minus'] = False

# 그림을 저장할 폴더
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)
    
import tensorflow as tf
import numpy as np

/home/shin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 32*32*3) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 32*32*3) / 255.0
y_train = y_train.astype(np.int32).reshape(-1)
y_test = y_test.astype(np.int32).reshape(-1)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]
m,n = X_train.shape
print(y_train.shape)
print(X_train.shape)

(45000,)
(45000, 3072)


In [2]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [3]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices] 
    return X_batch, y_batch

tf.layer.conv2d()의 kernel_initializer를 조절하는 것이 초기값을 어떻게 주느냐를 정할 수 있다.

- tf.contrib는 실험적인 부분의 내용이 들어있다.
- he_init = tf.contrib.layers.variance_scaling_initializer(factor=2.0, mode='FAN_IN', uniform=False) 

In [4]:
import tensorflow as tf
import numpy as np

height = 32
width = 32
channels = 3
n_inputs = height * width * 3

conv1_fmaps = 5
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "VALID"

pool2_fmap = conv1_fmaps

conv3_fmaps = 16
conv3_ksize = 5
conv3_stride = 1
conv3_pad = "VALID"

pool4_fmap = conv3_fmaps

conv5_fmaps = 32
conv5_ksize = 5
conv5_stride = 1
conv5_pad = "VALID"


n_fc1 = 84
n_outputs = 10

he_init = tf.contrib.layers.variance_scaling_initializer()

reset_graph()

with tf.name_scope("inputs"):
    X = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
    X_reshaped = tf.reshape(X, shape=[-1, height, width, channels])
    y = tf.placeholder(tf.int32, shape=[None], name="y")

with tf.name_scope("conv1"):
    conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize,
                         strides=conv1_stride, padding=conv1_pad,
                         activation=tf.nn.relu, kernel_initializer=he_init, name="conv1")

with tf.name_scope("pool2"):
    pool2 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="VALID")

with tf.name_scope("conv3"):
    conv3 = tf.layers.conv2d(pool2, filters=conv3_fmaps, kernel_size=conv3_ksize,
                         strides=conv3_stride, padding=conv3_pad,
                         activation=tf.nn.relu, kernel_initializer=he_init, name="conv3")

with tf.name_scope("pool4"):
    pool4 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="VALID")
    
with tf.name_scope("conv5"):
    conv5 = tf.layers.conv2d(pool4, filters=conv5_fmaps, kernel_size=conv5_ksize,
                            strides=conv5_stride, padding=conv5_pad,
                            activation=tf.nn.relu, kernel_initializer=he_init, name="conv5")
    
with tf.name_scope("pool_flat"):
    pool_flat = tf.reshape(conv5, shape=[-1, conv5_fmaps*1])

with tf.name_scope("fc1"):
    fc1 = tf.layers.dense(pool_flat, n_fc1, activation=tf.nn.relu, name="fc1")

with tf.name_scope("output"):
    logits = tf.layers.dense(fc1, n_outputs, name="output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [5]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [8]:
mse_summary = tf.summary.scalar('loss', loss)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [10]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [13]:
n_epochs = 100
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session(config=config) as sess:
    init.run()
    for epoch in range(n_epochs):
#         for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
#             sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "배치 데이터 정확도:", acc_batch, "검증 세트 정확도:", acc_val)

    acc_test = accuracy.eval(feed_dict={X: X_test,
                                        y: y_test})
    print("테스트 세트에서 최종 정확도:", acc_test)
        
    save_path = saver.save(sess, "./my_mnist_model")
    
    file_writer.close()

0 배치 데이터 정확도: 0.46 검증 세트 정확도: 0.455
1 배치 데이터 정확도: 0.54 검증 세트 정확도: 0.4842
2 배치 데이터 정확도: 0.47 검증 세트 정확도: 0.5108
3 배치 데이터 정확도: 0.54 검증 세트 정확도: 0.528
4 배치 데이터 정확도: 0.54 검증 세트 정확도: 0.5362
5 배치 데이터 정확도: 0.62 검증 세트 정확도: 0.5426
6 배치 데이터 정확도: 0.64 검증 세트 정확도: 0.5602
7 배치 데이터 정확도: 0.62 검증 세트 정확도: 0.5628
8 배치 데이터 정확도: 0.6 검증 세트 정확도: 0.5716
9 배치 데이터 정확도: 0.54 검증 세트 정확도: 0.5704
10 배치 데이터 정확도: 0.55 검증 세트 정확도: 0.5752
11 배치 데이터 정확도: 0.62 검증 세트 정확도: 0.5866
12 배치 데이터 정확도: 0.56 검증 세트 정확도: 0.5902
13 배치 데이터 정확도: 0.64 검증 세트 정확도: 0.6036
14 배치 데이터 정확도: 0.7 검증 세트 정확도: 0.5898
15 배치 데이터 정확도: 0.62 검증 세트 정확도: 0.5746
16 배치 데이터 정확도: 0.63 검증 세트 정확도: 0.5934
17 배치 데이터 정확도: 0.72 검증 세트 정확도: 0.5998
18 배치 데이터 정확도: 0.6 검증 세트 정확도: 0.5946
19 배치 데이터 정확도: 0.68 검증 세트 정확도: 0.5946
20 배치 데이터 정확도: 0.59 검증 세트 정확도: 0.6056
21 배치 데이터 정확도: 0.65 검증 세트 정확도: 0.6002
22 배치 데이터 정확도: 0.67 검증 세트 정확도: 0.6036
23 배치 데이터 정확도: 0.69 검증 세트 정확도: 0.5848
24 배치 데이터 정확도: 0.72 검증 세트 정확도: 0.6008
25 배치 데이터 정확도: 0.71 검증 세트 정확도: 0.5962
26 배치 데이터 정확도: 0.75 검증 세트 정

실험 결과

- 에폭값을 늘려도 오버피팅이 될뿐 정확도는 향상되지 않는다.
- initializer를 바꾸어도 크게 차이가 나지는 않는다.